In [1]:
import sys
from pathlib import Path
import fitz
from io import BytesIO
from PIL import Image
import io
import os

from modules.core.extractors.upstage_extractor import get_upstage_extractor
from modules.core.extractors.azure_extractor import get_azure_extractor

from modules.utils.image_rotation_utils import detect_and_correct_rotation
from modules.utils.config import load_env
load_env()

/opt/anaconda3/envs/rebate/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/nongshim/Desktop/Python/react_rebate/modules/core/extractors/gemini_extractor.py:17: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [3]:
# 프로젝트 루트를 Python 경로에 추가
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# PDF 파일 경로
pdf_path = Path("test/③ 株式会社大物-(1).pdf")
dpi=300
page_num = 9

In [5]:
# Upstage Extractor 생성
extractor = get_upstage_extractor(enable_cache=True)
upstage_text = extractor.extract_from_pdf_page(pdf_path, page_num, dpi=dpi)

✅ Upstage OCR 캐시 사용: test/③ 株式会社大物-(1)_Page9_upstage_ocr.json


In [4]:
# Azure Extractor 생성
azure_extractor = get_azure_extractor(enable_cache=True)
azure_text = azure_extractor.extract_from_pdf_page(pdf_path, page_num, dpi=dpi)

/opt/anaconda3/envs/rebate/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns-dt-team-test-edu.cognitiveservices.azure.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/rebate/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns-dt-team-test-edu.cognitiveservices.azure.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/rebate/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns-dt-team-test-edu.cognitiveservices.azure.com'. Adding certificate verification is strongly advised. See: https://urllib

✅ Azure OCR 완료 및 캐시 저장: test/③ 株式会社大物-(1)_Page9_azure_ocr.json


In [6]:
print(upstage_text)

伝票日付 伝票番号 伝票区分 商品コード JANコード 商品名 入数 単位 数量 単価 金額 未収率 未収円 現物数量 現物入金額 未収合計金額 直送日付 摘要 
20250203 3193567 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 3 105 7,560 0 17 0 0 1,224 0 玉出新今宮 
20250203 3193568 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 3 105 7,560 0 17 0 0 1,224 0 玉出天下茶 
20250203 3193569 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 4 105 10,080 0 17 0 0 1,632 0 玉出 今里 
20250203 3193570 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 5 105 12,600 0 17 0 0 2,040 0 玉出鶴見橋 
20250203 3193572 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 3 105 7,560 0 17 0 0 1,224 0 玉出千林店 
20250203 3193573 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 1 105 2,520 0 17 0 0 408 0 玉出 大池 
20250203 3193574 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 1 105 2,520 0 17 0 0 408 0 玉出 御� 
20250203 3193576 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 12 105 30,240 0 17 0 0 4,896 0 玉出恵美須 
20250203 3193577 11 047400052 8801043030694 辛カップラーメン 68G 24 ケース 1 105 2,520 0 17 0 0 408 0 玉出寝屋川 
20250203 3193583 11 

In [8]:
print(azure_text)

伝票日付 伝票番号 伝票区分 商品コード
JANコード
商品名
入数 単位 数量 単価
金額
未収率 未収円 現物数量 現物入金額| 未収合計金額 直送日付
摘要
20250203
3193567
11
047400052
8801043030694
辛カップラーメン 68G
24
5-7
3
105
7,560
0
17
0
0
1,224
0 玉出新今宮
20250203
3193568
11
047400052
8801043030694
辛カップラーメン 68G
24
4-7
3
105
7,560
0
17
0
0
1,224
0
玉出天下茶
20250203
3193569
11
047400052
8801043030694
辛カップラーメン 68G
24
5-7
4
105
10,080
0
17
0
0
1,632
0
玉出 今里
20250203
3193570
11
047400052
8801043030694
辛カップラーメン 68G
24
ケース
5
105
12.600
0
17
0
0
2.040
0 玉出鶴見橋
20250203
3193572
11
047400052
8801043030694
辛カップラーメン 68G
24
4-7
3
105
7,560
0
17
0
0
1.224
0
玉出千林店
20250203
3193573
11
047400052
8801043030694
辛カップラーメン 68G
24
T-7
1
105
2.520
0
17
0
0
408
0
玉出 大池
20250203
3193574
11
047400052
8801043030694
辛カップラーメン 68G
24
ケース
1
105
2.520
0
17
0
0
408
4.896
0
玉出 御陵
20250203
3193576
11
047400052
8801043030694
辛カップラーメン 68G
24
ケース
12
105
30,240
0
17
0
0
0
玉出恵美須
20250203
3193577
11
047400052
8801043030694
辛カップラーメン 68G
24
1
105
2,520
0
17
0
0
408
0
玉出寝屋川
20250203
3193583
11
047400052
880